In [1]:
from ast import Mult
import collections
from enum import unique
from re import T
import pandas as pd
import tensorflow as tf
from sqlalchemy import create_engine
import argparse
from datetime import datetime
import random
import os
from fixstring import fix_string

In [7]:
def write_tfrecords(tf_examples, filename):
  """Writes tf examples to tfrecord file, and returns the count."""
  with tf.io.TFRecordWriter(filename) as file_writer:
    length = len(tf_examples)
    progress_bar = tf.keras.utils.Progbar(length)
    for example in tf_examples:
      file_writer.write(example.SerializeToString())
      progress_bar.add(1)
    return length

In [14]:
def read__customer_data(sqlconn): 
    print('query non-duplicate customers...')
    query = f"""
      select 
        ID, Zip, TotalSales, TotalSavings, TotalVisits
      from
        Customer
      where
        ID not in (select ID from entity_map where cluster_score > 0.97);
      """
    customers_df = pd.read_sql(query, sqlconn)
    print('query duplicate entries')
    query = f"""
      select
        ID, Zip, TotalSales, TotalSavings, TotalVisits
      from
        Customer
      where
        ID in (select distinct canon_id from entity_map);"""

    customers_df = pd.concat([customers_df, pd.read_sql(query, sqlconn)])
    return customers_df

def generate_customer_examples(customers_df):
    examples = []
    print('Generate Customer Examples')
    progress_bar = tf.keras.utils.Progbar(len(customers_df))
    for _, customer in customers_df.iterrows():
        feature = {
            "user_id":
                tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[int(customer.ID)])),
           "zip_code":
                tf.train.Feature(
                    bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(str(customer.Zip))])),
            "total_visits":
                tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[int(customer.TotalVisits)])),
            "total_sales":
                tf.train.Feature(
                    float_list=tf.train.FloatList(value=[customer.TotalSales])),
            "total_savings":
                tf.train.Feature(
                    float_list=tf.train.FloatList(value=[customer.TotalSavings]))
        }
        tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
        examples.append(tf_example)
        progress_bar.add(1)
    return examples

In [15]:
customers_filename = "../../data/samples/customers.tfrecord"

print('Connect to Database...')
connect_str = 'mysql+mysqlconnector://' + 'sa' + ':' + 'rmssa512' + '@' + 'anand-dev' +'/' + 'RetailDB'
alchemy_engine = create_engine(connect_str)
sqlconn = alchemy_engine.connect()
print('Generating Datasets...')
customers_df = read__customer_data(sqlconn)
examples = generate_customer_examples(customers_df)
write_tfrecords(examples, customers_filename)

Connect to Database...
Generating Datasets...
query non-duplicate customers...
query duplicate entries
Generate Customer Examples
199034/199034 [==============================] - 495s 2ms/step


199034